# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [4]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations
import pandas as pd
from tqdm import tqdm_notebook
from datetime import datetime
%autosave 5
%matplotlib inline

Autosaving every 5 seconds


## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [5]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [6]:
historic_data = pd.read_csv('bitcoin_cash_price.csv')
historic_data.columns = historic_data.columns.str.lower()
historic_data['date'] = historic_data['date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))
historic_data.info()
historic_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        213 non-null    object 
 1   open        213 non-null    float64
 2   high        213 non-null    float64
 3   low         213 non-null    float64
 4   close       213 non-null    float64
 5   volume      213 non-null    object 
 6   market cap  213 non-null    object 
dtypes: float64(4), object(3)
memory usage: 11.8+ KB


,date,open,high,low,close,volume,market cap
0,2018-02-20,1543.27,1569.03,1414.35,1418.73,"820,947,000","26,199,800,000"
1,2018-02-19,1483.34,1553.81,1483.34,1534.77,"578,906,000","25,179,700,000"
2,2018-02-18,1552.10,1641.40,1428.49,1487.46,"907,873,000","26,344,200,000"
3,2018-02-17,1548.48,1568.64,1517.14,1551.39,"641,719,000","26,280,100,000"
4,2018-02-16,1373.16,1558.66,1369.68,1552.20,"961,010,000","23,302,000,000"
...,...,...,...,...,...,...,...
208,2017-07-27,417.10,460.97,367.78,385.48,"533,207",-
209,2017-07-26,407.08,486.16,321.79,365.82,"1,784,640",-
210,2017-07-25,441.35,541.66,338.09,406.90,"524,908",-
211,2017-07-24,412.58,578.89,409.21,440.70,"190,952",-


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [7]:
model_data=historic_data

## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [8]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [9]:
M.build()

In [10]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 883us/step - loss: 0.0089
Epoch 2/100
1/1 [==============================] - 0s 679us/step - loss: 0.0079
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0074
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0069
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 6/100
1/1 [==============================] - 0s 939us/step - loss: 0.0063
Epoch 7/100
1/1 [==============================] - 0s 925us/step - loss: 0.0060
Epoch 8/100
1/1 [==============================] - 0s 922us/step - loss: 0.0057
Epoch 9/100
1/1 [==============================] - 0s 844us/step - loss: 0.0055
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 11/100
1/1 [==============================] - 0s 872us/step - loss: 0.0051
Epoch 12/100
1/1 [==============================] - 0s 844us/step - loss: 0.0049
Epoch 13/100
1/1 [===========================

1/1 [==============================] - 0s 933us/step - loss: 4.5326e-06


We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [11]:
M.predict(denormalized=True)

array([1420.8762, 1103.4261, 1289.2073, 1435.4932, 1518.1206, 1437.2252,
       1371.762 ], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [12]:
M.evaluate()

{'mse': 0.0, 'rmse': 137.87, 'mape': 6.42}

Finally, we can now save the trained model on disk for later use.

In [13]:
M.save('bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [14]:
M = Model(path='bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [15]:
M.predict(denormalized=True)

array([1420.8762, 1103.4261, 1289.2073, 1435.4932, 1518.1206, 1437.2252,
       1371.762 ], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [16]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [17]:
M1.build()

In [18]:
M1.train()

In [21]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
A=1
B=11
C=0*7
results = []
for i in range(A, B):
    M1.train(model_data[C:(7*(40+i) + 7)])
    results.append(M1.evaluate())

In [22]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 136.43, 'mape': 6.43}
Week 42: {'mse': 0.0, 'rmse': 136.82, 'mape': 6.42}
Week 43: {'mse': 0.0, 'rmse': 136.72, 'mape': 6.42}
Week 44: {'mse': 0.0, 'rmse': 136.85, 'mape': 6.42}
Week 45: {'mse': 0.0, 'rmse': 136.25, 'mape': 6.44}
Week 46: {'mse': 0.0, 'rmse': 137.04, 'mape': 6.41}
Week 47: {'mse': 0.0, 'rmse': 137.44, 'mape': 6.41}
Week 48: {'mse': 0.0, 'rmse': 136.54, 'mape': 6.43}
Week 49: {'mse': 0.0, 'rmse': 136.2, 'mape': 6.44}
Week 50: {'mse': 0.0, 'rmse': 136.61, 'mape': 6.43}


In [23]:
M1.predict(denormalized=True)

array([1417.6177, 1100.044 , 1291.3823, 1437.8337, 1517.8467, 1439.7753,
       1367.7842], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [24]:
old_data = model_data[0*7:7*48 + 7]

In [25]:
new_data = model_data[0*7:7*49 + 7]

In [26]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [27]:
M2.build()
M1.train()

In [28]:
M2.predict(denormalized=True)

array([1413.505 , 1386.108 , 1445.4042, 1389.4191, 1421.062 , 1432.8567,
       1415.0548], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [29]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#
M3 = Model(data=new_data,
          variable = 'close',
          predicted_period_size=7)

In [30]:
M3.build()
M3.train()

In [31]:
M3.predict(denormalized=True)

array([1420.1062, 1103.3489, 1286.493 , 1434.1306, 1522.9209, 1437.8608,
       1369.8954], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.